In [1]:
! pip install dash plotly pandas numpy

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px

# Генерация синтетических OHLC данных
def generate_ohlc_data(days=30):
    """Генерация синтетических OHLC данных"""
    np.random.seed(42)
    
    start_date = datetime.now() - timedelta(days=days)
    dates = [start_date + timedelta(days=i) for i in range(days)]
    
    base_price = 100 + np.random.normal(0, 10)
    opens, highs, lows, closes, volumes = [], [], [], [], []
    
    for i in range(days):
        open_price = base_price if i == 0 else closes[i-1] * (1 + np.random.normal(0, 0.02))
        close_price = open_price * (1 + np.random.normal(0, 0.03))
        high_price = max(open_price, close_price) * (1 + abs(np.random.normal(0, 0.01)))
        low_price = min(open_price, close_price) * (1 - abs(np.random.normal(0, 0.01)))
        volume = int(np.random.normal(100000, 20000))
        
        opens.append(round(open_price, 2))
        closes.append(round(close_price, 2))
        highs.append(round(high_price, 2))
        lows.append(round(low_price, 2))
        volumes.append(volume)
    
    return pd.DataFrame({
        'Date': dates,
        'Open': opens,
        'High': highs,
        'Low': lows,
        'Close': closes,
        'Volume': volumes
    })

# Генерация данных
ohlc_data = generate_ohlc_data(60)  # 60 дней данных

# Инициализация приложения Dash
app = Dash(__name__)

# Макет дашборда
app.layout = html.Div([
    html.H1("Финансовый аналитический дашборд", style={'textAlign': 'center'}),
    
    # Панель управления
    html.Div([
        html.Div([
            html.Label("Тип графика:"),
            dcc.Dropdown(
                id='chart-type',
                options=[
                    {'label': 'Линейный', 'value': 'line'},
                    {'label': 'Свечной', 'value': 'candle'},
                    {'label': 'OHLC', 'value': 'ohlc'}
                ],
                value='candle',
                clearable=False
            ),
        ], style={'padding': '10px'}),
        
        html.Div([
            html.Label("Диапазон дат:"),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=ohlc_data['Date'].min(),
                max_date_allowed=ohlc_data['Date'].max(),
                start_date=ohlc_data['Date'].min(),
                end_date=ohlc_data['Date'].max()
            ),
        ], style={'padding': '10px'}),
        
        html.Div([
            html.Label("Скользящие средние:"),
            dcc.Checklist(
                id='ma-toggle',
                options=[
                    {'label': 'MA 7', 'value': 7},
                    {'label': 'MA 14', 'value': 14},
                    {'label': 'MA 30', 'value': 30}
                ],
                value=[]
            ),
        ], style={'padding': '10px'}),
        
        html.Div([
            html.Label("Индикаторы:"),
            dcc.Checklist(
                id='rsi-toggle',
                options=[
                    {'label': 'RSI (14)', 'value': 14}
                ],
                value=[]
            ),
        ], style={'padding': '10px'}),
    ], style={
        'width': '20%',
        'display': 'inline-block',
        'vertical-align': 'top',
        'border-right': '1px solid #ddd',
        'padding': '10px'
    }),
    
    # Основная область с графиками
    html.Div([
        dcc.Graph(id='price-chart'),
        dcc.Graph(id='volume-chart'),
        dcc.Graph(id='rsi-chart', style={'display': 'none'})
    ], style={'width': '78%', 'display': 'inline-block', 'padding': '10px'})
])

# Callback для обновления графиков
@app.callback(
    [Output('price-chart', 'figure'),
     Output('volume-chart', 'figure'),
     Output('rsi-chart', 'figure'),
     Output('rsi-chart', 'style')],
    [Input('chart-type', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('ma-toggle', 'value'),
     Input('rsi-toggle', 'value')]
)
def update_charts(chart_type, start_date, end_date, ma_values, rsi_values):
    """Обновление графиков на основе выбора пользователя"""
    # Фильтрация данных
    filtered_data = ohlc_data[
        (ohlc_data['Date'] >= pd.to_datetime(start_date)) & 
        (ohlc_data['Date'] <= pd.to_datetime(end_date))
    ].copy()
    
    # Создание ценового графика
    if chart_type == 'line':
        price_fig = px.line(filtered_data, x='Date', y='Close', title='Цена закрытия')
    elif chart_type == 'candle':
        price_fig = go.Figure(data=[go.Candlestick(
            x=filtered_data['Date'],
            open=filtered_data['Open'],
            high=filtered_data['High'],
            low=filtered_data['Low'],
            close=filtered_data['Close'],
            name='Цена'
        )])
        price_fig.update_layout(title='Свечной график')
    else:
        price_fig = go.Figure(data=[go.Ohlc(
            x=filtered_data['Date'],
            open=filtered_data['Open'],
            high=filtered_data['High'],
            low=filtered_data['Low'],
            close=filtered_data['Close'],
            name='Цена'
        )])
        price_fig.update_layout(title='OHLC график')
    
    # Добавление скользящих средних
    for window in ma_values:
        ma_col = f'MA_{window}'
        filtered_data[ma_col] = filtered_data['Close'].rolling(window=window).mean()
        price_fig.add_trace(go.Scatter(
            x=filtered_data['Date'],
            y=filtered_data[ma_col],
            name=f'MA {window}',
            line=dict(width=2)
        ))
    
    # График объема
    volume_fig = px.bar(filtered_data, x='Date', y='Volume', title='Объем торгов')
    
    # График RSI
    rsi_fig = go.Figure()
    rsi_style = {'display': 'none'}
    
    if rsi_values:
        window = rsi_values[0]
        delta = filtered_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        
        rsi_fig.add_trace(go.Scatter(
            x=filtered_data['Date'],
            y=rsi,
            name='RSI',
            line=dict(color='purple', width=2)
        ))
        rsi_fig.add_hline(y=30, line_dash="dash", line_color="red")
        rsi_fig.add_hline(y=70, line_dash="dash", line_color="red")
        rsi_fig.update_layout(title=f'RSI ({window} дней)', height=300)
        rsi_style = {'display': 'block'}
    
    # Общие настройки графиков
    for fig in [price_fig, volume_fig, rsi_fig]:
        fig.update_layout(
            hovermode='x unified',
            margin=dict(l=20, r=20, t=40, b=20),
            xaxis=dict(rangeslider=dict(visible=False)))
    
    price_fig.update_layout(height=500)
    volume_fig.update_layout(height=250)
    
    return price_fig, volume_fig, rsi_fig, rsi_style

# Запуск приложения
if __name__ == '__main__':
    app.run(debug=True)